In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shash\\fetch_assessment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shash\\fetch_assessment'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingModelConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    epoch: int
    batch_size: int
    look_back: int

In [26]:
from fetch_assessment.constants import *
from fetch_assessment.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        

    

    def get_training_model_config(self) -> TrainingModelConfig:
        config = self.config.model_training
        params=self.params.TrainingArguments
        create_directories([config.root_dir])

        traing_model_config = TrainingModelConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            epoch=params.epoch,
            batch_size=params.batch_size,
            look_back=params.look_back            
                    )

        return traing_model_config

In [88]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense,Dropout,Bidirectional,Conv1D,Reshape,MaxPooling1D,Flatten,SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import joblib

In [93]:
class ModelTrainer:
    def __init__(self, config: TrainingModelConfig):
        self.config = config

    
    def rmse_funct(self,y_true, y_pred):
        return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

    
    def training(self):
        import numpy as np

        # Load the data
        X_train = np.load(os.path.join(self.config.data_path,"X_train.npy"))
        y_train = np.load(os.path.join(self.config.data_path,"y_train.npy"))
        X_test = np.load(os.path.join(self.config.data_path,"X_test.npy"))
        y_test = np.load(os.path.join(self.config.data_path,"y_test.npy"))
        scaled_data = np.load(os.path.join(self.config.data_path,"scaled_data.npy"))
        scaler = joblib.load(os.path.join(self.config.data_path,"scaler.gz"))
        
        # X_loaded and Y_loaded are now numpy arrays with the same content and shape as the original X and Y
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True)

        model = Sequential()
        # CNN Layer
        model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(self.config.look_back, X_train.shape[2])))
        # Removing MaxPooling1D layer to maintain the sequence length
        # LSTM Layer
        model.add(LSTM(50, activation='relu', return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(50, activation='relu'))
        model.add(Dropout(0.2))
        # Output Layer
        model.add(Dense(1))

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Early stopping callback
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, restore_best_weights=True)

        # Train the model
        history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
        model.save(self.config.model_path)
        # # Predicting on the test set
        # y_pred = model.predict(X_test)
        # print(y_pred.shape,y_test.shape)
        # # Calculating RMSE
        # rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        # print("Test RMSE:", rmse)
        # # Reshape y_pred_scaled for inverse transformation
        # temp_shape = np.zeros((len(y_pred), scaled_data.shape[1]))
        # temp_shape[:, 0] = y_pred[:, 0]
        # y_pred = scaler.inverse_transform(temp_shape)[:, 0]

        # # Reshape y_test for inverse transformation
        # y_test_temp_shape = np.zeros((len(y_test), scaled_data.shape[1]))
        # y_test_temp_shape[:, 0] = y_test
        # y_test_rescaled = scaler.inverse_transform(y_test_temp_shape)[:, 0]

        # # Calculating RMSE on the rescaled data
        # rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred))
        # print("Test RMSE on original scale:", rmse)


In [94]:
config = ConfigurationManager()
model_trainer_config = config.get_training_model_config()
model_trainer_config = ModelTrainer(config=model_trainer_config)
model_trainer_config.training()

[2023-11-13 14:39:49,072: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-13 14:39:49,072: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-13 14:39:49,076: INFO: common: created directory at: artifacts]
[2023-11-13 14:39:49,076: INFO: common: created directory at: artifacts/model_training]


Epoch 1/100
8/8 [==============================] - 3s 49ms/step - loss: 0.2429 - val_loss: 0.1498
Epoch 2/100
8/8 [==============================] - 0s 12ms/step - loss: 0.0852 - val_loss: 0.0072
Epoch 3/100
8/8 [==============================] - 0s 12ms/step - loss: 0.0260 - val_loss: 0.0072
Epoch 4/100
8/8 [==============================] - 0s 14ms/step - loss: 0.0169 - val_loss: 0.0142
Epoch 5/100
8/8 [==============================] - 0s 14ms/step - loss: 0.0192 - val_loss: 0.0066
Epoch 6/100
8/8 [==============================] - 0s 13ms/step - loss: 0.0148 - val_loss: 0.0059
Epoch 7/100
8/8 [==============================] - 0s 15ms/step - loss: 0.0130 - val_loss: 0.0086
Epoch 8/100
8/8 [==============================] - 0s 13ms/step - loss: 0.0135 - val_loss: 0.0044
Epoch 9/100
8/8 [==============================] - 0s 13ms/step - loss: 0.0121 - val_loss: 0.0038
Epoch 10/100
8/8 [==============================] - 0s 13ms/step - loss: 0.0108 - val_loss: 0.0034
Epoch 11/100
8/8 [=